# Week 3 - 주간학습정리 - ML LifeCycle

![Machine Learning Development Lifecycle](https://miro.medium.com/v2/resize:fit:1400/format:webp/0*3I4P4pkL1xySQS9B.png)

## 깨달은 내용
* Machine Learning Lifecycle
* activation function
* loss function
* backpropagation
* parameter initializing
* Learning Rate Scheduling
* Encoder/Decoder
* LSTM
* Attention
* Transformer
* 한영 번역기 만들기


## Machine Learning Lifecycle
* 성능 좋은 모델이 일주일이 멀다하고 쏟아지는 현재 시점에서 Model Exploration이 더 중요해짐
* process
    1. Planning : 정량젹 목표 필요(정확도, F1점수, AUC) => 타당성 보고서
    1. Data Preparation : 수집 및 라벨링, 정리, 처리, 관리 => Cleaned Data
    1. Mode Engineering : 모델 아키텍처 구축, 메트릭 정의 , 학습/검증 => 모델압축, 결과해석
    1. Mode Evaluation : 프로덕트 확인, 법준수 체크, 견고성테스트 => 배포 결정
    1. Model Deployment : 크라우드/로컬 서버에 배포 => 서비스 런칭
    1. Monitoring and Maintenace : 잘 동작하나?, 목표 달성에 도움을 주나?, 고객만족도?, 성능은?


## activation function
* 목적
    * 입력의 선형결합에 비선형성 추가
    * 이게 없으면 아무리 많은 선형결합을 연결하여도 선형결합 하나로 간주할 수 있음
    * 복잡한 패턴을 학습할 수 있게 함
    * 비선형성이 필요없는 패턴을 찾을 경우는 사용하지 않을 수 있음
* 특징
    * vector -> vector 형식의 함수 형태 (다변수 다변량 함수)
    * R -> R 로의 함수를 element 별로 적용할 수 있으나 다 그렇지는 않음 (softmax는 element-wise함수가 아님)
    * 각 신경망 사이 사이에 추가함
* 종류
    * sigmoid()
        * element-wise 함수 : tensor의 경우 입력 shape과 출력 shape가 동일
        * 식 : $$\hat{y} = sigmoid(z) = \frac{1}{1+e^{-z}}$$
        * gradient
            * $\nabla_z\hat{y}$는 수학적으로는 야코비안이 맞지만 대각선 성분만 0이 아니기 때문에 실용적으로 vector로 간주함
            * 대응하는 element에만 영향을 미치는 element-wise 함수이기 때문에 대각선 성분만 있음
            $$
            \nabla_z\hat{y} = sigmoid(z)\odot(1-sigmoid(z)) = \hat{y}\odot(1-\hat{y})
            $$
    * tanh()
        * element-wise 함수 : tensor의 경우 입력 shape과 출력 shape가 동일
        * 식: $$\hat{y}=tanh(z)=\frac{sinh(z)}{cosh(z)} = \frac{e^z - e^{-z}}{e^z + e^{-z}}$$
        * gradient : $$\nabla_z\hat{y} = 1 - tanh(z)^2 = 1 - \hat{y}^2$$
    * ReLU()
        * element-wise 함수 : tensor의 경우 입력 shape과 출력 shape가 동일
        * 식 : $$\hat{y} = ReLU(z) =     
            \begin{cases} 
                z, & ( z \gt 0 ) \\
                0, & ( z \leq 0 ) 
            \end{cases} = max(0, z)$$
        * gradient : $$ \nabla_z\hat{y} = 
            \begin{cases} 
                1, & ( z \gt 0 ) \\
                0, & ( z \leq 0 ) 
            \end{cases} = max(0, z)$$
    * leaky ReLU()
        * ReLU()와 유사 
        * 식 : $max(ax, x)$
    * softmax()
        * element-wise 함수가 아님, vector -> vector 인 다변수 다변량 함수임
        * tensor가 입력되었을 경우는 마지막 차원을 입력 vector로 처리함
        * 입력 tensor와 출력 tensor의 shape은 동일
        * 식 : 
            $$
            \begin{align*}
            \hat{y} = softmax(z) 
            & = \frac{e^z}{1^Te^z} & (\text{ vector notation })\\
            & = \frac{e^{z_i}}{\sum_{j}e^{z_j}} & (\text{ element-wise notation })
            \end{align*}
            $$
        * gradient 
            * element-wise함수가 아니기 때문에 입력 벡타의 각 원소가 출력 벡터의 각 원소에게 영향을 미침
            * 야코비안 매트릭스임을 명확히 표현할 때는 $\nabla_z\hat{y}$ 대신에 $J_z(\hat{y})$ 라고 표기함
            * 10개의 class를 분류하는 문제에서 softmax gradient는 10x10 matrix 형태임
            $$
            \begin{align*}
            \nabla_z\hat{y} & = diag(\hat{y}) - \hat{y}\otimes\hat{y}^T & (\text{ vector notation }) \\
            \frac{\partial \hat{y}_i}{\partial z_j}  & = \hat{y}_i(\delta_{ij} - \hat{y}_j) & (\text{ element-wise notation }) \\
            & & \delta_{ij} \text{ ​는 크로네커 델타 (i=j일 때 1, 아닐 때 0)} 
            \end{align*}
            $$

### sigmoid() 특성 및 사용시 주의사항
* 특징 
    * vinishing gradent : 미분값이 (0, 1) 사이이기 때문에 층을 쌓아갈수록 점점 gradent 값이 줄어듬
    * non-zero centered 문제 : 함수 값 > 0 임으로 부호가 변경되지 않음, 파라미터가 특정 방향으로만 update 
* 주의 사항
    * 뉴럴 네트워크 마지막에만 사용
    * 중간층에는 sigmoid() 사용하지 말고 ReLU() 등으로 대체
    
### softmax() 특성 및 사용시 주의사항
* 주의사항
    * 계산과정에서 $e^x$을 하는데, 너무 큰수가 계산되어 overflow ('inf')가 발생할 수 있음
    * overflow를 해결하기 위해 모든 원소에서 max() 값을 뺀 후 softmax를 적용하는 방식이 가장 간단하고 효율적
* 특징
    * 수학적으로 각 원소에서 무슨 값을 뺀 후 softmax()를 계산해도 결과는 동일 함
    * max값을 뺀 후에는 $e^x$ 중 가장 큰 수가 1임으로 $\sum e^x \geq 0$ 이고 결과적으로 분모가 0이 될 수 없음
    * max()를 빼면 큰 음수가 나올 수 있는데, exp()를 적용하면 거의 0이나, 원래도 0이에 가까웠기 때문에 차이를 무시해도 됨


## loss function
* 목적
    * 모델이 예측한 값과 실제값 차이를 수치화
    * 이 수치가 작은 쪽으로 모든 파라미터를 튜닝하는 것이 학습의 핵심
    * 손실함수에 대한 각 파라미터의 편도함수를 계산하여 더 낮은 방향으로 파라미터를 수정해 나가는 방식으로 학습
* 특징
    * 입력벡터, 실벡터 -> 실수 형태의 함수 모양
    * 즉, 여러 feature로 구성된 하나의 point x와 label에 해당하는 y를 입력받아 scalar를 출력으로 내 보냄
    * scalar 로 모델의 성능을 확인 할 수있음 ( scalar가 작을 수록 성능이 우수 )
    * 손실함수는 모델의 마직막에 하나만 사용하며, 반드시 사용해야 함(필수)
    * 손실함수가 없으면, 경사하강법을 사용할 수 없음
* 종류
    * Mean Square Error
        * 오차의 제곱을 평균
    * Cross Entropy
        * 두 확률분포간의 차이를 측정
        * 모든 종류의 cross entropy를 의미하지만, 실무적으론 categorical cross-entropy를 의미
        * categorical cross-entropy : 다중 클래스 분류에 사용(3가지 이상 클래스), one-hot 인코딩
        * Sparse Categorical Cross-Entropy: 다중 클래스 분류용 (정수 라벨)
        * pytorch에서는 CrossEntroyLoss()를 제공하는데, softmax()와 합해진 함수임
        * multi-class(one-hot 인코딩)에서는 마지막에 softmax()없이 CrossEntroyLoss() 만 사용
    * Binary Cross Entropy
        * Cross Entropy의 한 종류로 이진 분류에 사용
        * 여러클래스를 동시에 선택하는 Multi-label 문제에서는 BCE를 class별로 적용하여 평균낸 것을 손실함수로 사용함
        * pytorch에서는 BCEWithLogitsLoss()를 제공하는데, sigmoid()와 합해진 함수임
        * 2진분류, multi-label 선택문에서는 마지막에선 sigmoid() 없이 BCEWithLogitsLoss() 만 사용

### Cross Entropy 
* BCE는 CE의 특수한 경우임, 서로 다른 것이 아님
    * CE ( or Categorical Cross-Entropy ) 는 muli-class에 분류에 활용
    * BCE( Binary Cross-Entropy )는 클래스 2개로 분류할 때 사용
* 계산식
    * class 의 갯수는 K
    * $y$는 클래스 하나만 표시한 label이며 one-hot encoding으로 표현 (하나의 원소만 1 이고 나머지는 0)
    * $\hat{y}$는 예측 확률로 각 클래스별 예측 확률의 합 $\sum_{k=1}^{K}\hat{y} = 1$ 임
    * CE공식에서 K가 2일때 $\sum$을 풀여서 전개하면 BCE공식이 됨
    $$
    \begin{align*}
    L_{CE} & = - \frac{1}{N} \sum_{i=1}^{N} \sum_{k=1}^{K} y_{ik}log(\hat{y}_{ik}), \quad \text{ N samples and K classes} \\
    & \quad y_{ik} = 
    \begin{cases} 
        1, & \text{ for one class } \\
        0, & \text{ others } 
    \end{cases} \\
    & \quad \sum_{k=1}^{K} \hat{y}_{ik} = 1, \hat{y}_{ik} \geq 0, \hat{y}_{ik} = probability \\
    \\
    L_{BCE} & = - \frac{1}{N} \sum_{i=1}^{N} [\underbrace{y_i log(\hat{y}_{i})}_{\text{ a class 100\%, [b class 0\% ]}}
    +\underbrace{(1-y_i)log(1-\hat{y}_{i})}_{\text{ a class 0\%, [b class 100\%]}}], 
    \quad \text{ N samples and 2 classes} \\
    & \quad 
    \end{align*}
    $$


### CrossEntroyLoss()
* softmax + Cross Entropy 가 합해진 것 (권장)
* 수치적 안정성
* 역전파 계산 효율성
* one-hot 인코딩으로 labeling 된 multi-class문제에서 손실함수로 활용
* 주의 사항 : NN 마지막에 activation함수 사용하지 말것

* Example of target with one-hot encoding

In [25]:
import numpy as np
import torch 
import torch.nn as nn

# Example of target with one-hot encoding
loss_fn = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
print(f'input=\n{input}')
random_index = np.random.randint(0, 5, size=3) # use np instead of torch to decrease memory usage
target = torch.eye(5)[random_index]
print(f'target=\n{target}')
loss = loss_fn(input, target)
print(f'loss={loss}')
loss.backward()
print(f'input.grad=\n{input.grad}')

input=
tensor([[-0.6048,  0.0601, -0.5818, -0.5541,  0.5132],
        [ 0.2571,  0.4849,  0.8432,  1.4265, -0.2677],
        [-0.6790, -0.2165, -0.6299, -0.2937, -0.1169]], requires_grad=True)
target=
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.]])
loss=1.5076652765274048
input.grad=
tensor([[-0.2921,  0.0802,  0.0422,  0.0434,  0.1262],
        [ 0.0424,  0.0532,  0.0762, -0.1969,  0.0251],
        [ 0.0486,  0.0771,  0.0510, -0.2619,  0.0852]])


* Example of target with class indices

In [ ]:
import torch 
import torch.nn as nn

# Example of target with class indices
loss_fn = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
print(f'input=\n{input}')
target = torch.empty(3, dtype=torch.long).random_(5)
print(f'target={target}')
loss = loss_fn(input, target)
print(f'loss={loss}')
loss.backward()
print(f'input.grad=\n{input.grad}')

input=
tensor([[-0.3054,  0.5108,  1.2683, -0.4611, -0.2587],
        [-0.1682,  2.5849,  0.8660,  0.9950, -0.0143],
        [-0.2764,  0.5265, -0.8201, -1.9561, -0.7972]], requires_grad=True)
target=tensor([4, 4, 3])
loss=2.8259646892547607
input.grad=
tensor([[ 0.0334,  0.0755,  0.1610,  0.0286, -0.2984],
        [ 0.0140,  0.2191,  0.0393,  0.0447, -0.3170],
        [ 0.0726,  0.1620,  0.0421, -0.3198,  0.0431]])


### BCEWithLogitsLoss()
* Sigmoid + Binary Cross-Entropy가 합쳐진 것(권장)
* 수치적 안정성
* 역전파 계산 효율성
* 2진 분류, multi-label 문제에서 손실함수로 사용
* 주의 사항 : NN 마지막에 activation함수 사용하지 말것

* Example of binary classification with logits

In [29]:
import torch
import torch.nn as nn

# Example of binary classification with logits
loss_fn = nn.BCEWithLogitsLoss()
z = torch.randn(3) * 3.
z.requires_grad_()
print(f'z={z}')
# sigmoid 중간 결과 확인 for debugging
y_hat = torch.sigmoid(z)
print(f"y_hat probabilities: {y_hat}")
y = torch.empty(3).random_(2)
print(f'y={y}')
loss = loss_fn(z, y)
print(f'loss={loss}')
loss.backward()
print(f'z.grad={z.grad}')

z=tensor([ 0.9366, -4.1564, -1.7502], requires_grad=True)
y_hat probabilities: tensor([0.7184, 0.0154, 0.1480], grad_fn=<SigmoidBackward0>)
y=tensor([1., 0., 0.])
loss=0.1688166707754135
z.grad=tensor([-0.0939,  0.0051,  0.0493])


* Example of multi-label classification

In [28]:
import torch
import torch.nn as nn

# Example of multi-label classification
loss_fn = nn.BCEWithLogitsLoss()
z = torch.randn(3, 5, requires_grad=True)
print(f'z=\n{z}')
# sigmoid 중간 결과 확인 for debugging
y_hat = torch.sigmoid(z)
print(f"y_hat probabilities=\n{y_hat}")
y = torch.empty(3, 5).random_(2)
print(f'y=\n{y}')
loss = loss_fn(z, y)
print(f'loss={loss}')
loss.backward()
print(f'input.grad=\n{z.grad}')

z=
tensor([[-0.2246,  0.1020, -0.9507,  1.5900, -0.7331],
        [-0.9135,  0.3956, -1.5659,  0.6002,  0.5753],
        [ 0.4001, -0.9471,  0.6100, -0.5276,  1.0691]], requires_grad=True)
y_hat probabilities=
tensor([[0.4441, 0.5255, 0.2787, 0.8306, 0.3245],
        [0.2863, 0.5976, 0.1728, 0.6457, 0.6400],
        [0.5987, 0.2795, 0.6479, 0.3711, 0.7444]], grad_fn=<SigmoidBackward0>)
y=
tensor([[0., 1., 0., 0., 0.],
        [0., 1., 1., 0., 1.],
        [0., 0., 0., 1., 0.]])
loss=0.8304862380027771
input.grad=
tensor([[ 0.0296, -0.0316,  0.0186,  0.0554,  0.0216],
        [ 0.0191, -0.0268, -0.0551,  0.0430, -0.0240],
        [ 0.0399,  0.0186,  0.0432, -0.0419,  0.0496]])


## backpropagation
* 벡터 미분 기본 성질
    $$
    \begin{align*} 
    & \nabla_\beta(a^T\beta) = a, \nabla_\beta(\beta^Ta) = a \\
    & \nabla_x(x^Tx) = 2x\\
    & \nabla_\beta(\beta^TA\beta) = (A+A^T)\beta, \quad 2A\beta \text{(when A is symetric)} \\
    & \nabla_\beta(\beta^T A^TA\beta) = 2A^TA\beta, \quad \because A^TA \text{ is symetric} \\
    \end{align*}
    $$

## parameter initializing

## Learning Rate Scheduling
* 라이브러리에서 제공하는 방식을 사용하기 보다 직접 결과를 관찰하며 튜닝해 가는 것이 현실적임
* initial warmup
    * 초기에 0에 가까운 값부터 시작
    * 점진적으로 linear 하게 증가 하면서 loss 추이를 관찰
* loss 감소가 정체되면 학습률을 더 줄이면 개선이 관찰됨
* 또 다시 loss 감소가 정체되면 학습률을 더 줄이면 또 다시 개선이 관찰됨
* 위 과정을 반복하여 적절한 학습률을 선택해 나감
* mini-batch 사이즈가 크면 학습률을 크게 사용함 ( 배치마다 한번 update가 되기 때문인 듯 )    

## Encoder/Decoder
* seq2seq
    * 글, 말하기, 날씨, 주식 가격 등 많은 시계열 데이터들이 sequence 형태
    * 입력 sequence를 출력 sequence로 변환하는 작업이 seq2seq라 함
    * 기계 번역, 문서 요약, 챗봇, 날씨 예측 등
    * 이를 수행하기 위해 보통 encoder/decoder 아키텍처를 사용함
    * 입력 데이터 형태 : ( batch_size, seq_length, feature_size )
        * batch_size : 배치 크기, 한번에 처리할 시퀀스 갯수
        * seq_length : 각 시퀀스 길이, 시퀀스 길이 만큼의 연속된 시간 스텝을 하나의 시퀀스로 간주
        * feature_size : 각 시점을 표현하는 피처 수, vector 차원
    * 예시
        * 자연어 문장
            - (32, 5, 128) : 배치는 32개 문장으로 구성, 각 문장은 5단어로 구성, 각 단어는 128차원 벡터로 표현
        * 주가 시계열 데이터
            - (64, 30, 4) : 배치는 64개 샘플, 샘플당 30일 데이터, 매일 시가/고가/저가/종가 데이터
        * IoT 센서 데이터
            - (16, 100, 10) : 배치는 16개 샘플, 샘플당 100시점, 시점당 10개 센서 값

* encoder/decoder architecture
    * encoding -> decoding 두 단계로 데이터를 처리하는 방식
    * LSTM 기반, Attention 기반, Self-Attention 기반(Transformer) 등 많은 곳에서 encoder/decoder 아키텍터를 활용하고 있음
    * Encoder
        * 입력 시퀀스를 하나로 압축 ( Many-to-One )
        * 최종적으로 context vector에 해당하는 hidden state로 전체 입력을 요약함
        * 최종 context vector를 Decoder가 활용
        * 입력 전체의 맥락을 파악하여 그 정보를 Decoder가 활용하는 구조
    * Decoder 
        * "시작" 입력 하나로 출력 시퀀스를 생성 ( One-to-Many )
        * "시작"은 특별한 token으로 등록하여 사용 보통, \<SOS\> start of sensense라 함
        * \<SOS\> 이후에는 이전 출력을 입력으로 하여 생성을 계속 함 (autoregressive)
        * \<EOS\> 가 생성될 때까지 계속 함
        * 그렇지만, 학습단계에서는 label을 인풋으로 사용 ( teacher forcing )
        

## LSTM

## Attention

## Transformer
* [임의의 퓨리에 급수 생성하여 학습한 후 Transformer로 예측](week03_transformer.ipynb)

## 한영 번역기 만들기
* 트랜스포머로 간단한 설계임에도 patameter 갯수 8천만개
    * embedding vector size : 512
    * header 갯수 : 8개
    * MHA layer : 3개층
* 학습 시간
    * training data 9만5천개
    * PC(Ultra 9 275HX, RTX 5080)
        * 파라미터 8,000만개 : CPU-1시간30분/epoch, GPU-3분/epoch, 쓰로틀링GPU-30분/epoch
        * 파라미터 80만개   : CPU-30분/epoch, GPU-1~2분/epoch, 쓰로틀링GPU-15분/epoch
    * Colab(free)
        * 파라미터 8,000만개 : CPU-15시간/epoch, T4-15분/epoch
        * 파라미터 80만개   : CPU-5시간/epoch, T4-5분/epoch
    * apple(M4)
        * 파라미터 8,000만개 : CPU-1시간20분/epoch, MPS-50분/epoch (메모리 부족으로 종료)
        * 파라미터 80만개   : CPU-30분/epoch, MPS-4분/epoch
* OS 별 안정성
    * windows11(WSL) : 학습 실행하면 바로 시스템 오류 발생하고 시스템 리부팅
    * Ubuntu 25.04 : 실행가능, 메모리 부족으로 오류 발생시에도 해당 python process만 종료
    * macOS Sequoia 15 : 실행가능, 메모리 부족으로 오류 발생시에도 해당 python process만 종료
* [AI Hub](https://www.aihub.or.kr/) 라는 유용한 사이트 발견
    * AI허브란?
        - "AI 허브는 과학기술정보통신부와 한국지능정보사회진흥원이 운영하는 국가 AI 개발 지원 플랫폼으로, AI 기술 및 제품·서비스 개발에 필요한 AI 인프라(AI 데이터, AI SW API, 컴퓨팅 자원)를 대한민국 국민이면 누구나(연구자, 기업, 공공기관 등) 활용 할 수 있도록 제공 및 지원합니다."